# <font color="#418FDE" size="6.5" uppercase>**Exception Hierarchy**</font>

>Last update: 20251221.
    
By the end of this Lecture, you will be able to:
- Describe the structure of the built-in exception hierarchy and the roles of BaseException and Exception. 
- Select appropriate built-in exception types to signal different categories of errors in your code. 
- Use isinstance checks to handle groups of related built-in exceptions in try-except blocks. 


## **1. BaseException and System Exits**

### **1.1. BaseException and Exception**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_A/image_01_01.jpg?v=1766295205" width="250">



>* BaseException is the root of all exceptions
>* Exception covers everyday errors normal code should handle

>* Everyday recoverable errors subclass Exception in Python
>* Serious interpreter signals subclass BaseException, not Exception

>* Application code usually catches Exception and subclasses
>* BaseException-only signals should propagate, not be swallowed



In [ ]:
#@title Python Code - BaseException and Exception

# Show difference between BaseException and Exception hierarchy usage.
# Demonstrate catching Exception versus catching BaseException carefully.
# Print which exceptions are caught by different except blocks.

# This function raises a normal Exception subclass using invalid integer conversion.
# It represents everyday errors that application code should usually handle.
def cause_normal_error():
    value = "not_a_number"
    return int(value)


# This function simulates a system level signal using KeyboardInterrupt instance.
# We manually raise it to show that it inherits directly from BaseException.
def cause_system_like_signal():
    raise KeyboardInterrupt("Simulated Ctrl+C press")


print("Catching only Exception subclasses, normal error is handled.")
try:
    cause_normal_error()
except Exception as exc:
    print("Handled normal error:", type(exc).__name__)


print("Now catching BaseException, system signal is also handled.")
try:
    cause_system_like_signal()
except BaseException as exc:
    print("Handled base level signal:", type(exc).__name__)



### **1.2. SystemExit and KeyboardInterrupt**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_A/image_01_02.jpg?v=1766295255" width="250">



>* SystemExit and KeyboardInterrupt tell Python to stop
>* They act as control signals, not bugs

>* SystemExit is intentional, carries exit status info
>* KeyboardInterrupt comes from user, stops program execution

>* Let these exceptions usually terminate the program
>* Optionally catch them for cleanup, then re-raise



In [ ]:
#@title Python Code - SystemExit and KeyboardInterrupt

# Demonstrate SystemExit and KeyboardInterrupt as special control signal exceptions.
# Show how SystemExit can stop a script intentionally and return a status code.
# Show how KeyboardInterrupt can be caught briefly then re-raised for clean stopping.

import time
import sys

print("Starting demo script showing special stop exceptions.")
print("First, we trigger an intentional SystemExit now.")

try:
    raise SystemExit(2)
except SystemExit as exit_error:
    print("Caught SystemExit with status code value.", exit_error.code)

print("Continuing after handling SystemExit intentionally.")
print("Now imagine a long task that user might interrupt.")

try:
    print("Pretend user presses Control plus C during waiting.")
    time.sleep(1.0)
    raise KeyboardInterrupt()
except KeyboardInterrupt:
    print("Caught KeyboardInterrupt, performing quick cleanup then re-raising.")
    raise KeyboardInterrupt()



### **1.3. Avoid Catching BaseException**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_A/image_01_03.jpg?v=1766295310" width="250">



>* Avoid catching BaseException in normal code
>* Catching it can block important shutdown signals

>* Catching BaseException can block user interrupts
>* It also breaks clean shutdowns in automation

>* Catch specific Exception types, not BaseException
>* Let system-level stop signals pass through safely



In [ ]:
#@title Python Code - Avoid Catching BaseException

# Demonstrate why catching BaseException is dangerous in simple scripts.
# Show KeyboardInterrupt being swallowed by an overly broad exception handler.
# Then show safer handling using Exception instead of BaseException.

import time


def unsafe_loop_with_baseexception():
    print("Starting unsafe loop, press Ctrl+C to interrupt.")
    try:
        for second in range(3):
            print(f"Unsafe loop second {second}, still running.")
            time.sleep(0.5)
    except BaseException as error:
        print("Caught BaseException, ignoring shutdown signal.")
        print(f"Error type was {type(error).__name__}.")


def safe_loop_with_exception():
    print("Starting safe loop, press Ctrl+C to interrupt.")
    try:
        for second in range(3):
            print(f"Safe loop second {second}, still running.")
            time.sleep(0.5)
    except Exception as error:
        print("Caught normal Exception, handling expected problems.")
        print(f"Error type was {type(error).__name__}.")


print("First, running unsafe loop that catches BaseException.")
unsafe_loop_with_baseexception()
print("Now, running safer loop that catches Exception only.")
safe_loop_with_exception()



## **2. Choosing Builtin Errors**

### **2.1. Type Versus Value Errors**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_A/image_02_01.jpg?v=1766295358" width="250">



>* Type errors mean the argument’s kind is wrong
>* They signal violating the function’s input contract

>* ValueError: right type, but invalid content
>* Example: impossible date or expired passport details

>* Consistent type versus value errors clarify failures
>* They improve debugging, error handling, and integration



In [ ]:
#@title Python Code - Type Versus Value Errors

# Demonstrate difference between TypeError and ValueError using simple functions.
# Show type error when argument category is completely wrong for operation.
# Show value error when argument type is correct but specific content is invalid.

from datetime import date


def days_until_birthday(birthday_string):
    """Calculate days until birthday, expecting MM-DD string format."""
    if not isinstance(birthday_string, str):
        raise TypeError("birthday_string must be string, for example '07-04'.")

    try:
        month_text, day_text = birthday_string.split("-")
    except ValueError as split_error:
        raise ValueError("Birthday string must match 'MM-DD' pattern.") from split_error

    month_number = int(month_text)
    day_number = int(day_text)

    if not 1 <= month_number <= 12:
        raise ValueError("Month value must be between 1 and 12 inclusive.")

    if not 1 <= day_number <= 31:
        raise ValueError("Day value must be between 1 and 31 inclusive.")

    today_date = date.today()
    current_year = today_date.year

    birthday_this_year = date(current_year, month_number, day_number)
    if birthday_this_year < today_date:
        birthday_this_year = date(current_year + 1, month_number, day_number)

    difference_days = (birthday_this_year - today_date).days
    return difference_days


print("Correct type and value example result:")
print(days_until_birthday("12-25"))

print("\nCorrect type but invalid value example:")
try:
    days_until_birthday("13-40")
except ValueError as value_problem:
    print("Caught ValueError:", value_problem)

print("\nCompletely wrong type example:")
try:
    days_until_birthday(1234)
except TypeError as type_problem:
    print("Caught TypeError:", type_problem)



### **2.2. Indexing and Lookup Errors**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_A/image_02_02.jpg?v=1766295404" width="250">



>* Use IndexError for out-of-range sequence positions
>* Use KeyError when requested mapping key missing

>* Use IndexError when sequences are too short
>* Use KeyError when requested configuration key missing

>* Use IndexError and KeyError for targeted fallbacks
>* Specific errors enable informative, recoverable failure handling



In [ ]:
#@title Python Code - Indexing and Lookup Errors

# Demonstrate index errors with lists and key errors with dictionaries.
# Show how different exceptions describe different missing data situations.
# Help choose correct error type for indexing and lookup operations.

sensor_readings = [72.5, 73.0, 71.8]  # Example Fahrenheit sensor readings list.
config_settings = {"language": "en", "theme": "dark"}  # Example configuration dictionary.

print("Sensor readings list:", sensor_readings)  # Display current sensor readings list.
print("Config settings dictionary:", config_settings)  # Display current configuration settings.

try:
    print("Fourth reading:", sensor_readings[3])  # This index is outside valid range.
except IndexError as error:
    print("Caught IndexError:", error)  # Explains missing position in sequential collection.

try:
    print("Timeout setting:", config_settings["timeout"])  # This key does not exist.
except KeyError as error:
    print("Caught KeyError:", error)  # Explains missing key in mapping collection.

print("Use IndexError for bad positions, KeyError for unknown names.")  # Summarize key takeaway.



### **2.3. Runtime and System Errors**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_A/image_02_03.jpg?v=1766295443" width="250">



>* Distinguish logic bugs from runtime or system failures
>* Catch these only to clean up or log

>* Recursion limit errors show unsustainable recursive design
>* Let them surface, then refactor algorithmic approach

>* Use system-level exceptions for graceful shutdown actions
>* Otherwise let them terminate and rely on restarts



In [ ]:
#@title Python Code - Runtime and System Errors

# Demonstrate runtime and system related exceptions handling briefly.
# Show recursion limit error and keyboard interrupt graceful shutdown example.
# Emphasize avoiding catching every possible base exception type blindly.

import sys
import time


def recursive_depth_counter(current_depth, maximum_depth):
    if current_depth >= maximum_depth:
        print("Reached safe depth limit, stopping recursion manually.")
        return current_depth
    return recursive_depth_counter(current_depth + 1, maximum_depth)


def demonstrate_recursion_limit():
    print("Current recursion limit value is:", sys.getrecursionlimit())
    try:
        print("Starting deep recursion demonstration, please wait briefly.")
        recursive_depth_counter(0, 1000)
    except RecursionError as recursion_problem:
        print("Caught RecursionError, algorithm design likely needs improvement.")
        print("Error message was:", recursion_problem)


def demonstrate_keyboard_interrupt():
    print("Starting long running loop, press Ctrl+C to interrupt.")
    try:
        for second_counter in range(10):
            print("Working second", second_counter + 1, "of ten total seconds.")
            time.sleep(1)
    except KeyboardInterrupt:
        print("KeyboardInterrupt received, performing graceful shutdown actions.")
    finally:
        print("Cleanup finished, program will exit safely now.")


if __name__ == "__main__":
    demonstrate_recursion_limit()
    demonstrate_keyboard_interrupt()



## **3. Catching Related Exceptions**

### **3.1. Grouping Exceptions with Tuples**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_A/image_03_01.jpg?v=1766295491" width="250">



>* Group related exceptions instead of many handlers
>* Use tuples to handle similar file errors

>* Treat tuples as families of related exceptions
>* Handle all invalid input errors with one handler

>* Group related exceptions into clear failure categories
>* Update tuples to keep handling consistent, maintainable



In [ ]:
#@title Python Code - Grouping Exceptions with Tuples

# Demonstrate grouping related exceptions using tuples in a simple example.
# Show how one handler catches several input related error types together.
# Keep code beginner friendly and output short and readable.

# Define a helper function that converts text into a positive integer value.
# Several different exceptions may occur during this conversion process.
# We will group these exceptions using a tuple in one handler.
# This keeps our error handling logic short, clear, and consistent.

from typing import Tuple, Union, Any

# Create a tuple that groups related conversion exception classes together.
# These exceptions all represent invalid or unusable user input values.
# ValueError handles bad digits, TypeError handles wrong input types.
# OverflowError handles numbers that are far too large for integers.

conversion_errors: Tuple[type[BaseException], ...] = (ValueError, TypeError, OverflowError)

# Define a function that tries converting a text value into a positive integer.
# It returns either the converted integer or a helpful error message string.
# The except block uses our grouped tuple of related exception classes.
# One handler now covers several different but related error situations.

def to_positive_int(text: Any) -> Union[int, str]:

    try:
        number: int = int(text)
        if number <= 0:
            raise ValueError("Number must be positive for this simple example.")
        return number
    except conversion_errors as error:
        return f"Invalid input detected, details: {error.__class__.__name__}."

# Prepare several test values that will trigger different exception situations.
# These include letters, huge numbers, negative numbers, and a None value.
# All of them should be handled by the same grouped except block.
# Only the valid positive integer should convert successfully without errors.

samples: list[Any] = ["42", "abc", "9999999999999999999999999", -5, None]

# Loop through each sample and show how the grouped handler behaves.
# We print both the original value and the conversion result for clarity.
# This demonstrates that different exceptions share one common response.
# The output remains short while still clearly illustrating the concept.

for value in samples:
    result = to_positive_int(value)
    print(f"Input value {value!r} produced result: {result}")



### **3.2. Ordering Except Clauses**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_A/image_03_02.jpg?v=1766295537" width="250">



>* Exception handlers run in top-to-bottom order
>* Put specific handlers before general ones, especially with isinstance

>* Specific handlers must come before general handlers
>* Correct order enables different responses to errors

>* Layer exception handlers from specific to general
>* Correct ordering keeps code predictable, clear, maintainable



In [ ]:
#@title Python Code - Ordering Except Clauses

# Demonstrate exception handler ordering with specific and general handlers.
# Show how a broad handler can hide specific handlers.
# Then fix ordering to handle exceptions more precisely.

class DataError(Exception):
    """Custom base error for data related problems."""

class MissingFieldError(DataError):
    """Specific error for missing required data field."""


def process_record_bad(record):
    """Process record with incorrect handler ordering for demonstration."""
    try:
        if "name" not in record:
            raise MissingFieldError("Required field 'name' is missing.")
    except DataError as error:
        print("Bad ordering caught DataError handler:", type(error).__name__)
    except MissingFieldError as error:
        print("Bad ordering caught MissingFieldError handler:", type(error).__name__)


def process_record_good(record):
    """Process record with correct handler ordering for clarity."""
    try:
        if "name" not in record:
            raise MissingFieldError("Required field 'name' is missing.")
    except MissingFieldError as error:
        print("Good ordering caught MissingFieldError handler:", type(error).__name__)
    except DataError as error:
        print("Good ordering caught DataError handler:", type(error).__name__)


bad_record = {"age": 30, "height_inches": 70}

print("Running function with bad handler ordering now.")
process_record_bad(bad_record)

print("Running function with good handler ordering now.")
process_record_good(bad_record)



### **3.3. Avoiding Bare Except**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_A/image_03_03.jpg?v=1766295592" width="250">



>* Bare except catches everything, hiding error meaning
>* It obscures serious failures and complicates debugging

>* Catch only expected, related exceptions using types
>* Let unexpected exceptions propagate to fail loudly

>* Catch expected exceptions; log and handle them clearly
>* Let unexpected errors surface to reveal serious problems



In [ ]:
#@title Python Code - Avoiding Bare Except

# Demonstrate why bare except handlers are dangerous and should be avoided.
# Show safer handling using specific exception types and isinstance checks.
# Compare outputs from bare except and explicit exception handling.

import math

# This function uses a bare except handler, which hides important problems.
def unsafe_divide_and_sqrt(numerator, denominator):
    try:
        result = numerator / denominator
        return math.sqrt(result)
    except:  # Bare except hides unexpected serious errors.
        print("Unsafe handler caught something, returning default value.")
        return -1.0


# This function handles only expected numeric related exceptions explicitly.
def safe_divide_and_sqrt(numerator, denominator):
    try:
        result = numerator / denominator
        return math.sqrt(result)
    except Exception as exc:
        if isinstance(exc, (ZeroDivisionError, ValueError, TypeError)):
            print("Safe handler caught expected numeric problem.")
            return -1.0
        raise


print("Calling unsafe_divide_and_sqrt with denominator zero.")
print("Result:", unsafe_divide_and_sqrt(10, 0))

print("\nCalling safe_divide_and_sqrt with denominator zero.")
print("Result:", safe_divide_and_sqrt(10, 0))



# <font color="#418FDE" size="6.5" uppercase>**Exception Hierarchy**</font>


In this lecture, you learned to:
- Describe the structure of the built-in exception hierarchy and the roles of BaseException and Exception. 
- Select appropriate built-in exception types to signal different categories of errors in your code. 
- Use isinstance checks to handle groups of related built-in exceptions in try-except blocks. 

In the next Lecture (Lecture B), we will go over 'Raising And Handling'